# Ad Campaign Performance: Exploratory Data Analysis

`The Ad Campaign Performance project conducts an A/B hypothesis testing on an Ad campaign. During the Ad campaign, SmartAd, a mobile first advertising company is running an online Ad for brand Lux. SmartAd company runs two versions of an Ad, a creative Ad which was presented to the exposed group and a dummy Ad which was presented to the control group. The main objective of the project is to determine through A/B hypothesis testing whether the creative Ad results to a significant lift in the brand awareness. This notebook conducts preprocessing and exploratory data analysis on the data generated during the campaign.`

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy